In [2]:
import h5py
import pandas as pd
import numpy as np

In [ ]:
file_dir = r"T:\60day-TEMP\Hana\results_alloc_20250602.h5"
myh5 = h5py.File(file_dir, "r")

# parcels_urbansim.txt

In [ ]:

myh5.keys()
[print (i) for i in myh5['base'].keys()]

# myh5['base']['cities']['axis0'][:]
# get h5py as datafarme
# df = pd.DataFrame()
# for key in myh5['base'].keys():
#     df[key] = myh5['base'][key][:]

building_sqft_per_job
building_types
buildings
cities
control_hcts
controls
development_constraints
development_template_components
development_templates
employment_controls
employment_sector_group_definitions
employment_sector_groups
employment_sectors
fazes
gridcells
household_controls
household_relocation_rates
households
households_events
households_zone_control_hct_events
job_relocation_rates
jobs
land_use_types
mpds
parcels
persons
schools
subregs
target_vacancies
targets
travel_data
zone_control_hcts
zones
zoning_heights


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [5]:
myh5['base']['parcels']['axis1'][:]

array([      1,       2,       3, ..., 1329926, 1329927, 1329928],
      shape=(1329928,))

In [151]:
myh5['base']['households']['axis1'][:]

array([      1,       2,       3, ..., 1736132, 1736133, 1736134],
      shape=(1736134,))

In [146]:
myh5['base']['households']['block0_items'][:]

array([b'tenure', b'puma', b'building_id', b'income', b'persons',
       b'workers', b'children', b'previous_building_id', b'age_of_head'],
      dtype='|S20')

In [143]:
myh5['base']['households']['block1_items'][:]

array([b'is_inmigrant'], dtype='|S12')

In [9]:
myh5['base']['buildings'].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values']>

In [44]:
len(myh5['base']['buildings']['block0_values'][:][0])

11

In [43]:
myh5['base']['schools']['block1_values'][:][0]

array([128,   5, 149, ..., 148,  98,  46], shape=(112717,), dtype=uint8)

In [122]:


def get_col_data(myh5, key, block, col_name):
    myarray = myh5['base'][key][f'block{block}_items'][:]
    col_list = [x.decode('utf-8') for x in myarray]
    col_index = col_list.index(col_name)

    return myh5['base'][key][f'block{block}_values'][:][:,col_index]

def calculate_jobs_by_sector(df, parcel_df, sector_id_list, emp_type):
    """
    Calculate the number of jobs for a given list of sector IDs and return a DataFrame.
    """
    df_filtered = df[df['sector_id'].isin(sector_id_list)]
    df_grouped = df_filtered.groupby('parcelid').sum()[['number_of_jobs']].reset_index()
    df_grouped.rename(columns={'number_of_jobs': emp_type}, inplace=True)
    
    parcel_df = parcel_df.merge(df_grouped, on='parcelid', how='left')
    parcel_df.fillna({emp_type: 0}, inplace=True)

    return parcel_df

In [ ]:
parcel_df = pd.DataFrame()
# parcelid
# Assume parcel IDs from the index?
parcel_df['parcelid'] = myh5['base']['parcels']['axis1'][:]

# xcoord_p
parcel_df['xcoord_p'] = get_col_data(myh5, 'parcels', 0, 'x_coord_sp')

# ycoord_p
parcel_df['ycoord_p'] = get_col_data(myh5, 'parcels', 0, 'y_coord_sp')

# sqft_p
parcel_df['sqft_p'] = get_col_data(myh5, 'parcels', 0, 'parcel_sqft')

# taz_p
parcel_df['taz_p'] = get_col_data(myh5, 'parcels', 1, 'zone_id')

# lutype_p
parcel_df['lutype_p'] = get_col_data(myh5, 'parcels', 1, 'land_use_type_id')


# Household data is available at the building level
df_building = pd.DataFrame()
df_building['building_id'] = myh5['base']['buildings']['axis1'][:]
df_building['parcelid'] = get_col_data(myh5, 'buildings', 0, 'parcel_id')


# Note for Hana:
# Some households have building_id == -1 (N=5

# # hh_p
df_hh = pd.DataFrame(get_col_data(myh5, 'households', 0, 'building_id'), columns=['building_id'])
# Exclude households without valid building_id
df_hh = df_hh[df_hh['building_id'] >= 0 ].reset_index(drop=True)
df_hh = df_hh.groupby('building_id').size().reset_index(name='household_count')

# Merge household count data to building data
df_building_hh = df_building.merge(df_hh, on='building_id', how='left')
df_hh_tot = df_building_hh.groupby('parcelid').sum()[['household_count']].reset_index().rename(columns={'household_count': 'hh_p'})

parcel_df = parcel_df.merge(df_hh_tot, on='parcelid', how='left')
parcel_df.fillna({'hh_p': 0}, inplace=True)

# students

# not able to access the "category" field in block1, so not sure how to classify schools...

# df_school = pd.DataFrame()
# df_school['codist'] = get_col_data(myh5, 'schools', 2, 'codist')
# df_school['student_count2023'] = get_col_data(myh5, 'schools', 2, 'student_count2023')
# df_school['parcelid'] = get_col_data(myh5, 'schools', 2, 'parcel_id')


# stugrd_p
# stuhgh_p
# stuuni_p

# Jobs
df_jobs = pd.DataFrame()

df_jobs["sector_id"] = get_col_data(myh5, 'jobs', 1, 'sector_id')
df_jobs["building_id"] = get_col_data(myh5, 'jobs', 1, 'building_id')
df_jobs["number_of_jobs"] = get_col_data(myh5, 'jobs', 0, 'number_of_jobs')
df_jobs["home_based_status"] = get_col_data(myh5, 'jobs', 1, 'home_based_status')

# Remove home_based_status jobs
df_jobs = df_jobs[df_jobs['home_based_status'] == 0].reset_index(drop=True)
df_jobs = df_jobs.merge(df_building, on='building_id', how='left')

df_jobs = df_jobs.groupby(['sector_id','parcelid']).sum()[['number_of_jobs']].reset_index()

# empedu_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [8,13], "empedu_p")

# empfoo_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [10], "empfoo_p")

# empgov_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [12], "empgov_p")

# empind_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [3, 4], "empind_p")

# empmed_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [9], "empmed_p")

# empofc_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [7], "empofc_p")

# empret_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [5], "empret_p")

# empsvc_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [11], "empsvc_p")

# empoth_p
parcel_df = calculate_jobs_by_sector(df_jobs, parcel_df, [1, 2], "empoth_p")

# emprsc_p
# FIXME: make sure this is correct
parcel_df["emprsc_p"] = 0

# emptot_p
parcel_df["emptot_p"] = parcel_df[[
    'empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
    'empofc_p', 'empret_p', 'empsvc_p', 'empoth_p', 'emprsc_p'
]].sum(axis=1)

# ppricdyp
parcel_df['ppricdyp'] = get_col_data(myh5, 'parcels', 1, 'parking_price_daily')

# pprichrp
parcel_df['pprichrp'] = get_col_data(myh5, 'parcels', 1, 'parking_price_hourly')

# parkdy_p
parcel_df['parkdy_p'] = get_col_data(myh5, 'parcels', 1, 'parking_space_daily')

# parkhr_p
parcel_df['parkhr_p'] = get_col_data(myh5, 'parcels', 1, 'parking_space_hourly')

# Parks data is zeroed out in Soundcast anyway
# aparks
parcel_df['aparks'] = 0

# nparks
parcel_df['nparks'] = 0





In [128]:
parcel_df

,parcelid,xcoord_p,ycoord_p,sqft_p,taz_p,lutype_p,hh_p,empedu_p
0,1,1292255.125,162728.625000,85802.593750,1019,22,0.0,0.0
1,2,1291832.250,164041.750000,0.000000,1018,26,0.0,0.0
2,3,1291594.625,164048.671875,6000.066406,1018,18,0.0,0.0
3,4,1291539.625,164050.171875,7200.083984,1018,18,0.0,0.0
4,5,1291479.375,164042.390625,6068.041504,1018,26,0.0,0.0
...,...,...,...,...,...,...,...,...
1329923,1329924,1444396.250,305434.312500,93749.859375,2365,17,0.0,0.0
1329924,1329925,1310124.125,305445.000000,49211.039062,2538,17,0.0,0.0
1329925,1329926,1294709.250,305556.187500,14059.324219,2526,17,0.0,0.0
1329926,1329927,1306089.375,288830.875000,10867.518555,2565,17,0.0,0.0


# hh_and_persons.h5

In [ ]:
df_hh = pd.DataFrame(get_col_data(myh5, 'households', 0, 'building_id'), columns=['building_id'])
# # Exclude households without valid building_id
# df_hh = df_hh[df_hh['building_id'] >= 0 ].reset_index(drop=True)
df_hh['hhno'] = myh5['base']['households']['axis1'][:]

df_hh['hhsize'] = get_col_data(myh5, 'households', 0, 'persons')
df_hh['workers'] = get_col_data(myh5, 'households', 0, 'workers')
df_hh['children'] = get_col_data(myh5, 'households', 0, 'children')
df_hh['tenure'] = get_col_data(myh5, 'households', 0, 'tenure')

# Map hownrent
df_hh['hownrent'] = df_hh['tenure'].map({
    1: 1, # 'own',
    2: 2, # 'rent',
    3: 3, # 'other'
    4: 9 # 'missing'
})

# Merge parcel ID from building data
df_hh = df_hh.merge(df_building[['building_id', 'parcelid']], on='building_id', how='left')

In [164]:
df_hh['tenure'].value_counts()

tenure
1    733950
3    686112
2    297466
4     18606
Name: count, dtype: int64